In [ ]:
# Global Env

openai_api  = ""    # openai-api-key
model       = ""    # path of model,pth file
model_conf  = ""    # path of config,json format


In [ ]:
# clone vits repo
!git clone https://github.com/kagura114/vits.git vits

#  USE A NEW ENV AND RUN!!!
#  FOLLOWING SCRIPT WILL REMOVE YOUR CURRENT TRNSORFLOW INSTALLATION

In [ ]:
# install requirements
# python==3.9

%pip install --upgrade pip
%pip uninstall -y tensorflow numba
%pip install Cython==0.29.21 \
librosa==0.8.0               \
matplotlib==3.3.1            \
numpy==1.23                  \
unidecode==1.3.4             \
pyopenjtalk==0.2.0           \
jamo==0.4.1                  \
pypinyin==0.44.0             \
jieba==0.42.1                \
protobuf==3.19.0             \
cn2an==0.5.17                \
inflect==6.0.0               \
eng_to_ipa==0.0.2            \
ko_pron==1.3                 \
indic_transliteration==2.3.37\
num_thai==0.0.5              \
opencc==1.1.1                \
openai    

%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

# Remove "#" and install if you meet trouble
# %pip install scipy==1.5.2


In [ ]:
#build monotonic_align
%cd vits/monotonic_align
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ../../


In [ ]:
#initialize vits

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

%cd vits

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

%cd ..

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

hps = utils.get_hparams_from_file(model_conf)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint(model, net_g, None)

def mix(_):
    stn_tst = get_text(_, hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [ ]:
mix("こんにちは")

In [ ]:
#Loat ChatGPT and perform the test
import openai
sys_prompt = "You are my assistant, please answer my question and reply with Japanese, no emojis nor special characters, also use kana instead of kanji.\n"
user = "Hello"
openai.api_key = openai_api
response_r = openai.Completion.create(
  model="text-davinci-003",
  prompt= f"{sys_prompt}{user}Answer:",
  temperature=0.9,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.6,
  stop=[":"]
)
print(response_r)

response = str(response_r['choices'][0]['text'])
print(response)
mix(response)

In [ ]:
# then load as a function
def qa(x:str):
    response_r = openai.Completion.create(
    model="text-davinci-003",
    prompt= f"{sys_prompt}{x}Answer:",
    temperature=0.9,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.6,
    stop=[":"]
    )
    response = str(response_r['choices'][0]['text'])
    print(response)
    mix(response)

In [ ]:
qa("I wanna get a new video card, can you give me some advice")